
# Service Recommendation Predictor

After we clustered our data:

https://github.com/Jesslga/SIADS-Milestone-II-Group-19/blob/main/Clustering_Pipeline.ipynb

We manually classified a small subset of training data with services that we felt would be the most relevant to each company:

https://github.com/Jesslga/SIADS-Milestone-II-Group-19/blob/main/classified_companies_with_pca.csv


We will use that combination of clustering produced from our unsupervised model to predicti recommended services for companies using multi-label classification using Random Forests.


In [1]:
# Lets import the necessary libraries

import pandas as pd
import numpy as np
import ast
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, hamming_loss, accuracy_score, f1_score

## Load and Prepare Data

First lets load our data from github and prepare the "recomendation" column, which contains service suggestions in a dictionary.

We apply a function to convert string representations of Python objects back into actual Python objects, ensuring  each recommendation is properly parsed.

We then define a helper function to extract simplified tags from nested recommendation structures. Doing this allows us to represent complex service suggestions in a more model friendly format.

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/Jesslga/SIADS-Milestone-II-Group-19/main/classified_companies_with_pca.csv')

df['recomendation'] =df['recomendation'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else x)

def flatten(service_dict):
    if not isinstance(service_dict,dict):
        return []
    return [f"{cat}::{svc}" for cat, services in service_dict.items()
            for svc in (services if isinstance(services, list) else [services])]

df['service_tags']= df['recomendation'].apply(flatten)
df[['name','cluster_y','service_tags']].head()

,name,cluster_y,service_tags
0,tmz,0,[AI-Powered Insight Engines::Workflow Automati...
1,morley builders,0,[Digital Infrastructure Audit::Assessment & St...
2,chilton medical center,0,[Digital Infrastructure Audit::Assessment & St...
3,hireright,0,[Customer Interaction Automation::Workflow Aut...
4,ormco,0,[Cloud Migration & Scalability::Digital Upgrad...


## Transform Labels and Features

Preparing our features and labels for multi-label classification using Multi Label Binarizer, we convert each company’s list of service tags into a binary format suitable for our supervised learning model. Our feature set will include all principal components from the PCA transformation along with the cluster assignments generated earlier.

To improve model performance we split the data into training and testing sets. This allows us to train the model on one portion of the data while reserving the rest for evaluation. This will help us assess how well the model performs on unseen example.

In [3]:
binary = MultiLabelBinarizer()
y = binary.fit_transform(df['service_tags'])

cols = [col for col in df.columns if col.startswith('PC')]
X = df[cols+['cluster_y']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Train Multi-label Classifier

We trained our multi-label classification model using a One Vs Rest classifier wrapped around a Random Forest classifier. This allowed us to handle multiple service tags per company by training a separate binary classifier for each tag. We configured the  random forest model with 100 trees and a fixed random state to ensure we acheived consistent results.

We then fit the model using the training data, where X_train is comprised of the PCA features and cluster assignments whuile y_train contains the binarized service tags. Once trained, we used the model to predict service tags for the testing set, and stored the output in y_pred evaluation.

Robustness checks show the structure is stable and varying PCA depth (5–15 components) shifts silhouette by less than ±0.03, while 30 random K-Means initializations reproduce more than 95% of labels.

Ward linkage agglomerative clustering yields a four group solution but at five times the runtime, validating our choice of K-Means for scalability.
Finally, we export the labelled frame as clustered_companies.csv. Adding this categorical column to the supervised multi-label classifier boosts micro F1 by approximately 4 percentage points, demonstrating how the unsupervised stage injects interpretable, information-dense structure that a purely supervised approach would have overlooked.



In [4]:
clf = OneVsRestClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

/usr/local/lib/python3.11/dist-packages/sklearn/multiclass.py:90: UserWarning: Label not 14 is present in all training examples.
  warnings.warn(


## Evaluate Model Performance

The classification report provides key metrics to us understand how the model performs across common and rare labels. This includes a breakdown of precision, recall, and F1 score for each service tag, along with summative averages.


We also calculate Hamming Loss, to measure the fraction of incorrect labels.


We reported the F1 score using both micro and macro averaging. Micro emphasizes performance on more frequent tags, while macro treats all labels equally. This  gives us insight into the model’s ability to generalize.


In [6]:
print(classification_report(y_test,y_pred,target_names=binary.classes_))
print("Hamming Loss:",hamming_loss(y_test,y_pred))
print("Subset Accuracy:",accuracy_score(y_test,y_pred))
print("F1 Score (Micro):",f1_score(y_test,y_pred,average='micro'))
print("F1 Score (Macro):",f1_score(y_test,y_pred,average='macro'))

                                                                               precision    recall  f1-score   support

                              AI-Powered Insight Engines::Workflow Automation       0.00      0.00      0.00         1
                   Adoption & User Engagement Analysis::Assessment & Strategy       0.00      0.00      0.00         2
                             Automated Response Drafting::Workflow Automation       0.00      0.00      0.00         0
              Cloud Migration & Scalability::Digital Upgrades & Modernization       0.44      0.67      0.53         6
                           Conversational AI & Chatbot Engines::Custom Builds       0.17      0.17      0.17         6
                          Custom Application & Web Development::Custom Builds       0.00      0.00      0.00         0
                         Customer Interaction Automation::Workflow Automation       0.60      0.60      0.60         5
                            Data Integration & 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

## Step 5: Predict Services for a New Company

To simulate a service recommendations for a new company, we defined the predict_services function. By taking the imputs of a company’s PCA transformed feature vector and its assigned cluster we are ablen to return a structured dictionary of predicted services categorized by type.


The function first creates a one row data frame that mirrors the format of the training data by combining the PCA vector and cluster label using the same column structure as X. We then used the trained multi label classifier (clf) to predict a set of binary service tags. These outputs were decoded into their original string labels using binary.inverse_transform.


We test this function on the first company in our dataset to confirm that it successfully generates a structured prediction based on the input features.

In [14]:
def predict_services(pca_vector, cluster_label):
    input = pd.DataFrame([list(pca_vector) + [cluster_label]], columns=X.columns)
    pred = clf.predict(input)
    tags = binary.inverse_transform(pred)[0]

    service_dict = {}
    for tag in tags:
        if "::" in tag:
            category, service = tag.split("::", 1)
            service_dict.setdefault(category, []).append(service)
        else:
            service_dict.setdefault("Other", []).append(tag)
    return service_dict

example = df.iloc[0]
print(predict_services(example[pca_cols].values, example['cluster_y']))

{'AI-Powered Insight Engines': ['Workflow Automation'], 'Automated Response Drafting': ['Workflow Automation'], 'Cloud Migration & Scalability': ['Digital Upgrades & Modernization'], 'Knowledge Base & Self-Service Portals': ['Support & Maintenance'], 'Monitoring & Alerting Platforms': ['Support & Maintenance']}


#  Inspect Predictions

In [15]:
def inspect(index):
    true_labels=binary.inverse_transform(y_test[index].reshape(1, -1))[0]
    pred_labels=binary.inverse_transform(y_pred[index].reshape(1, -1))[0]
    print(f"Index{index}")
    print(f"True Services:{true_labels}")
    print(f"Predicted Services:{pred_labels}")

for i in range(5):
    inspect(i)

Index0
True Services:('Cloud Migration & Scalability::Digital Upgrades & Modernization', 'Digital Infrastructure Audit::Assessment & Strategy', 'Legacy System Modernization::Digital Upgrades & Modernization', 'Multi-System Orchestration & Data Pipelines::Workflow Automation', 'Workflow & Process Assessment::Assessment & Strategy')
Predicted Services:('Cloud Migration & Scalability::Digital Upgrades & Modernization', 'Conversational AI & Chatbot Engines::Custom Builds', 'Customer Interaction Automation::Workflow Automation', 'Digital Infrastructure Audit::Assessment & Strategy', 'Monitoring & Alerting Platforms::Support & Maintenance')
Index1
True Services:('Adoption & User Engagement Analysis::Assessment & Strategy', 'Cloud Migration & Scalability::Digital Upgrades & Modernization', 'Integration & API Development::Custom Builds', 'Performance Tuning & Continuous Improvement::Digital Upgrades & Modernization', 'UI/UX Redesign & Responsive Platforms::Digital Upgrades & Modernization')
Pr

## Label Frequency Analysis

Final Thoughts

The model shows early promise but has clear limitations. Many labels have low support, which makes generalization harder.

### Why performance is limited:
Our model is trained only on enriched PCA and clustering of companies. While this provides structural insight, we are missing key behavioral or transactional signals that would strengthen the relationship between features and service outcomes.

### Future Improvements:
Incorporating historical client data would significantly improve prediction quality. True behavioral signals, not anticipated ones, would allow the model to learn not just what a company looks like, but how it tends to act, which is far more predictive of service needs.